<a href="https://colab.research.google.com/github/s1kcode/lib/blob/main/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install bertviz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.6/157.6 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 9.1 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
from transformers import BertForMaskedLM, BertConfig,  BertTokenizer
from huggingface_hub import hf_hub_download
from bertviz import head_view, model_view

hf_hub_download(repo_id="bert-base-uncased", filename="tf_model.h5")
hf_hub_download(repo_id="bert-base-uncased", filename="tokenizer.json")
hf_hub_download(repo_id="bert-base-uncased", filename="tokenizer_config.json")
hf_hub_download(repo_id="bert-base-uncased", filename="vocab.txt")
hf_hub_download(repo_id="bert-base-uncased", filename="config.json")

tf_model.h5:   0%|          | 0.00/536M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

'/root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/1dbc166cf8765166998eff31ade2eb64c8a40076/config.json'

In [3]:
# COPY FROM ABOVE
repo_id = "/root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/1dbc166cf8765166998eff31ade2eb64c8a40076/"

tokenizer = BertTokenizer.from_pretrained(repo_id)
model = BertForMaskedLM.from_pretrained(repo_id, from_tf=True)

All TF 2.0 model weights were used when initializing BertForMaskedLM.

Some weights of BertForMaskedLM were not initialized from the TF 2.0 model and are newly initialized: ['cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
def unmask(masked_input):
    inputs = tokenizer(masked_input, return_tensors="pt")
    outputs = model(**inputs, output_attentions=True)
    logits = outputs.logits
    attentions = outputs.attentions
    mask_token_index = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]
    predicted_token_id = logits[0, mask_token_index].argmax(axis=-1)
    return attentions, tokenizer.decode(predicted_token_id)

def split_sentence(sentence):
  for d in ".,'":
    sentence = sentence.replace(d, f" {d} ")
    tokens = ["<start>"] + sentence.split() + ["<end>"]
  return tokens

In [5]:
_, res = unmask("Never [MASK] give you up.")
res

'gonna'

In [6]:
phrase = """I once had a friend called Mickey.
            We've been together for a long time.
            I called him my brother.
            One day all of a sudden he disappeared.
            All our town was looking for him, but without any luck.
            Jack, Mary and their dog called Betsy were searching in the forest.
            We called him so many times.
            The disappeared person's name was [MASK]."""
_, res = unmask(phrase)
res

'mickey'

In [7]:
phrase = """I once had a friend called [MASK].
            We've been together for a long time.
            I called him my brother.
            One day all of a sudden he disappeared.
            All our town was looking for him, but without any luck.
            Jack, Mary and their dog called Betsy were searching in the forest.
            We called him so many times.
            The disappeared person's name was Mickey."""
_, res = unmask(phrase)
res

'mickey'

In [8]:
_, res = unmask("Mary's Father has Five daughters. They are Nana, Nene, Nini, Nono, and [MASK].")
res

'mary'

In [9]:
_, res = unmask("Two plus two equals [MASK].")
res

'one'

In [10]:
phrase = "The rabbit quickly hopped. The turtle slowly crawled."
attentions, _ = unmask(phrase)

head_view(attentions, split_sentence(phrase))

<IPython.core.display.Javascript object>